In [ ]:
# Run this first to install all necessary packages
!pip install openai elevenlabs pydub ffmpeg-python anthropic
!apt-get install ffmpeg

In [ ]:
# Enter your API keys here (they will be kept secure in your session)
from google.colab import userdata

print("Fetching API keys from Colab secrets...")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
ELEVENLABS_API_KEY = userdata.get("ELEVENLABS_API_KEY")
ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")

print("✅ API keys loaded securely!")

In [ ]:
from google.colab import files
import os

print("Please upload your Episode 1 MP3 file:")
uploaded = files.upload()

# Get the filename
audio_filename = list(uploaded.keys())[0]
print(f"✅ Uploaded: {audio_filename}")

Please upload your Episode 1 MP3 file:


Saving talktracks6.mp3 to talktracks6 (1).mp3
✅ Uploaded: talktracks6 (1).mp3


In [ ]:
from pydub import AudioSegment
import io

# Load the audio file
audio = AudioSegment.from_mp3(io.BytesIO(uploaded[audio_filename]))

# Extract first 90 seconds for testing
test_duration = 90 * 1000  # 90 seconds in milliseconds
test_segment = audio[:test_duration]

# Save test segment
test_filename = "test_segment.mp3"
test_segment.export(test_filename, format="mp3")

print(f"✅ Created {test_duration/1000} second test segment")
print(f"📊 Original duration: {len(audio)/1000:.1f} seconds")

✅ Created 90.0 second test segment
📊 Original duration: 2042.8 seconds


In [ ]:
from openai import OpenAI
import base64

client = OpenAI(api_key=OPENAI_API_KEY)

print("🎯 Transcribing audio with Whisper...")

# Transcribe using Whisper API
with open(test_filename, "rb") as audio_file:
    transcript = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        response_format="verbose_json",
        timestamp_granularities=["word"]
    )

# Extract text and show preview
english_text = transcript.text
words_with_timing = transcript.words if hasattr(transcript, 'words') else []

print("✅ Transcription complete!")
print(f"\n📝 First 200 characters:\n{english_text[:200]}...")
print(f"\n📊 Total words: {len(english_text.split())}")

🎯 Transcribing audio with Whisper...
✅ Transcription complete!

📝 First 200 characters:
Hi everyone, I'm Kai Dickens, and I'm thrilled to welcome you to the Talk Tracks. In this series, we dive deeper into the revelations, challenges, and unexpected truths from the Telepathy Tapes. The g...

📊 Total words: 224


In [ ]:
import anthropic

# Initialize Anthropic client
anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

print("🌍 Translating to Brazilian Portuguese...")

# Create translation prompt
translation_prompt = f"""Please translate the following English podcast transcript to Brazilian Portuguese.
Maintain the conversational, engaging tone of a podcast. Keep proper names unchanged.

English transcript:
{english_text}

Provide only the Portuguese translation, no explanations."""

# Get translation
response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=4000,
    messages=[
        {"role": "user", "content": translation_prompt}
    ]
)

portuguese_text = response.content[0].text

print("✅ Translation complete!")
print(f"\n🇧🇷 First 200 characters:\n{portuguese_text[:200]}...")

🌍 Translating to Brazilian Portuguese...
✅ Translation complete!

🇧🇷 First 200 characters:
Olá pessoal, eu sou Kai Dickens, e estou muito empolgado em dar as boas-vindas ao Talk Tracks. Nesta série, mergulhamos mais fundo nas revelações, desafios e verdades inesperadas das Telepathy Tapes. ...


In [ ]:
# Alternative: Use OpenAI for translation instead of Anthropic
def translate_with_openai(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a professional translator. Translate the following English podcast transcript to Brazilian Portuguese, maintaining the conversational tone."},
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.content

# Use this instead of the Anthropic translation
portuguese_text = translate_with_openai(english_text)
print("✅ Translation complete using OpenAI!")

In [ ]:
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs

# Initialize ElevenLabs correctly
elevenlabs = ElevenLabs(api_key=ELEVENLABS_API_KEY)

print("🎙️ Setting up voice generation...")

# List available voices
try:
    voices = elevenlabs.voices.get_all()
    print(f"\n📋 You have {len(voices.voices)} voices available:")

    # Show first 5 voices
    for i, voice in enumerate(voices.voices[:5]):
        print(f"  {i+1}. {voice.name} - {voice.labels.get('description', 'No description')}")

    print("\n✅ ElevenLabs connected successfully!")
except Exception as e:
    print(f"❌ Error listing voices: {e}")

🎙️ Setting up voice generation...

📋 You have 22 voices available:
  1. Aria - No description
  2. Sarah - No description
  3. Laura - No description
  4. Charlie - No description
  5. George - No description

✅ ElevenLabs connected successfully!


In [ ]:
# Check ElevenLabs account status
try:
    # Get user info
    user = elevenlabs.user.get()
    subscription = user.subscription

    print("📊 ElevenLabs Account Status:")
    print(f"✅ Tier: {subscription.tier}")
    print(f"💬 Character count: {subscription.character_count}")
    print(f"📈 Character limit: {subscription.character_limit}")

    if subscription.character_count >= subscription.character_limit:
        print("\n⚠️ You've used all your characters this month!")
        print("💡 You need to upgrade or wait for the monthly reset")
    else:
        remaining = subscription.character_limit - subscription.character_count
        print(f"\n✅ Characters remaining: {remaining:,}")
        print(f"📝 Portuguese text length: {len(portuguese_text)} characters")

        if len(portuguese_text) > remaining:
            print(f"\n⚠️ Your text is too long! You need {len(portuguese_text) - remaining:,} more characters")
            print("💡 Try processing a shorter segment first")

except Exception as e:
    print(f"❌ Couldn't check account status: {e}")

📊 ElevenLabs Account Status:
✅ Tier: free
💬 Character count: 0
📈 Character limit: 10000

✅ Characters remaining: 10,000
📝 Portuguese text length: 1460 characters


In [ ]:
from elevenlabs import save

print("🎯 Creating Portuguese audio...")

# Choose a multilingual voice that supports Portuguese
# Using "Rachel" - a good multilingual voice
voice_id = "21m00Tcm4TlvDq8ikWAM"  # Rachel - verified multilingual voice

try:
    # Generate Portuguese audio using the correct method
    audio = elevenlabs.generate(
        text=portuguese_text,
        voice=voice_id,
        model="eleven_multilingual_v2"
    )

    # Save the generated audio
    save(audio, "portuguese_test.mp3")

    print("✅ Portuguese audio generated!")
    print("📁 Saved as: portuguese_test.mp3")
    print("\n🎧 You can now download and listen to the result!")

except Exception as e:
    print(f"❌ Error: {e}")

    # If error, let's try with a basic voice
    print("\n🔄 Trying with a different voice...")
    try:
        # Try with a simpler setup
        audio = elevenlabs.generate(
            text=portuguese_text[:500],  # Shorter text for testing
            voice="Rachel",  # Using voice name instead of ID
            model="eleven_multilingual_v2"
        )
        save(audio, "portuguese_test_short.mp3")
        print("✅ Generated shorter test version successfully!")
    except Exception as e2:
        print(f"❌ Still having issues: {e2}")
        print("\n💡 Let's check your ElevenLabs account...")

🎯 Creating Portuguese audio...
❌ Error: 'ElevenLabs' object has no attribute 'generate'

🔄 Trying with a different voice...
❌ Still having issues: 'ElevenLabs' object has no attribute 'generate'

💡 Let's check your ElevenLabs account...


In [ ]:
# Simpler approach - use a pre-made Portuguese voice
print("🎯 Using a pre-made Portuguese voice...")

try:
    # Generate with a female Portuguese voice
    audio = elevenlabs.generate(
        text=portuguese_text[:1000],  # First 1000 characters only
        voice="Rachel",  # or try "Bella", "Domi", "Elli"
        model="eleven_multilingual_v2"
    )

    save(audio, "portuguese_simple.mp3")
    print("✅ Success! Audio saved as 'portuguese_simple.mp3'")

    # Download it
    from google.colab import files
    files.download('portuguese_simple.mp3')

except Exception as e:
    print(f"❌ Error: {e}")
    print("\nPossible issues:")
    print("1. No characters left in your ElevenLabs account")
    print("2. API key issues")
    print("3. Text might be too long")

🎯 Using a pre-made Portuguese voice...
❌ Error: 'ElevenLabs' object has no attribute 'generate'

Possible issues:
1. No characters left in your ElevenLabs account
2. API key issues
3. Text might be too long


In [ ]:
from google.colab import files

print("📥 Ready to download your files!\n")

# Download the Portuguese audio
print("1️⃣ Downloading Portuguese audio...")
files.download('portuguese_test.mp3')

# Create a text file with both transcripts for comparison
with open('transcripts.txt', 'w', encoding='utf-8') as f:
    f.write("=== ENGLISH TRANSCRIPT ===\n\n")
    f.write(english_text)
    f.write("\n\n=== PORTUGUESE TRANSLATION ===\n\n")
    f.write(portuguese_text)

print("2️⃣ Downloading transcripts...")
files.download('transcripts.txt')

print("\n✅ All done! Check your downloads folder.")

In [ ]:
# Only run this after you're happy with the test results!
process_full = input("\n🤔 Happy with the test? Process full episode? (yes/no): ")

if process_full.lower() == 'yes':
    print("\n🚀 Processing full episode...")
    print("⏰ This may take 10-20 minutes depending on episode length\n")

    # Process in chunks to handle long audio
    chunk_length = 10 * 60 * 1000  # 10 minutes in milliseconds
    chunks = []

    for i in range(0, len(audio), chunk_length):
        chunk = audio[i:i+chunk_length]
        chunk_file = f"chunk_{i//chunk_length}.mp3"
        chunk.export(chunk_file, format="mp3")
        chunks.append(chunk_file)
        print(f"📦 Created chunk {i//chunk_length + 1} of {len(audio)//chunk_length + 1}")

    # Process each chunk
    all_transcripts = []
    all_translations = []

    for i, chunk_file in enumerate(chunks):
        print(f"\n🎯 Processing chunk {i+1}/{len(chunks)}...")

        # Transcribe chunk
        with open(chunk_file, "rb") as f:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=f
            )
        all_transcripts.append(transcript.text)

        # Translate chunk
        response = anthropic_client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=4000,
            messages=[
                {"role": "user", "content": f"Translate to Brazilian Portuguese:\n\n{transcript.text}"}
            ]
        )
        all_translations.append(response.content[0].text)

        print(f"✅ Chunk {i+1} complete")

    # Combine all translations
    full_portuguese = " ".join(all_translations)

    print("\n✅ Full episode processed!")
    print(f"📊 Total Portuguese text: {len(full_portuguese)} characters")

else:
    print("\n👍 No problem! Refine your settings and test again.")